In [1]:
import os

# Define the new working directory relative to the current working directory
# or use an absolute path
new_working_directory = os.path.join(os.getcwd(), '..', '..', '..') # Set to MLAgentBenhc
print("New working directory: ", new_working_directory)

# Change the current working directory
os.chdir(new_working_directory)

print("New Working Directory:", os.getcwd())

New working directory:  c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench\MLAgentBench_v2\agents\Voyager\..\..\..
New Working Directory: c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench


In [2]:
from MLAgentBench_v2.agents.agent import Agent
import numpy as np 
import json
import logging 
import os
import re
import subprocess
from pathlib import Path
import shutil
import time 
import types
import copy

In [3]:
# Instantiate an environment

from types import SimpleNamespace
from MLAgentBench_v2.environment import Environment

args = SimpleNamespace(
    task='home-data-for-ml-course',
    task_type='kaggle',
    log_dir='logs/house-price-testing_voyager-v4_gpt4_v3_execute_script_non_blocking_windows_friendly',
    work_dir='workspace',
    max_steps=50,
    max_time=18000,
    device=0,
    python='/home/user/micromamba/envs/autogpt/bin/python',
    interactive=False,
    resume=None,
    resume_step=0,
    agent_type='SimpleAssistantAgent',
    # llm_name='gpt-3.5-turbo-1106',
    # fast_llm_name='gpt-3.5-turbo-1106',
    # edit_script_llm_name='gpt-3.5-turbo-1106',
    llm_name='gpt-4-1106-preview',
    fast_llm_name='gpt-4-1106-preview',
    edit_script_llm_name='gpt-4-1106-preview',
    edit_script_llm_max_tokens=4000,
    agent_max_steps=50,
    actions_remove_from_prompt=[],
    actions_add_to_prompt=[],
    no_retrieval=False,
    valid_format_entires=None,
    max_steps_in_context=3,
    max_observation_steps_in_context=3,
    max_retries=4,
    langchain_agent='zero-shot-react-description'
)

env = Environment(args)

No module named 'helm'
Could not load CRFM API key crfm_api_key.txt.
[Errno 2] No such file or directory: 'claude_api_key.txt'
Could not load anthropic API key claude_api_key.txt.
Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/house-price-testing_voyager-v4_gpt4_v3_execute_script_non_blocking_windows_friendly', work_dir='workspace', max_steps=50, max_time=18000, device=0, python='/home/user/micromamba/envs/autogpt/bin/python', interactive=False, resume=None, resume_step=0, agent_type='SimpleAssistantAgent', llm_name='gpt-4-1106-preview', fast_llm_name='gpt-4-1106-preview', edit_script_llm_name='gpt-4-1106-preview', edit_script_llm_max_tokens=4000, agent_max_steps=50, actions_remove_from_prompt=[], actions_add_to_prompt=[], no_retrieval=False, valid_format_entires=None, max_steps_in_context=3, max_observation_steps_in_context=3, max_retries=4, langchain_agent='zero-shot-react-description')
Preparing task home-data-for-ml-cour

In [4]:
class CurriculumAgent(Agent):
    def __init__(self, env, completed_tasks=[], failed_tasks=[]):
        super().__init__(env)
        self.completed_tasks = completed_tasks
        self.failed_tasks = failed_tasks

        # The crux is a Q&A process
        # Problem with this approach is you still have to deal with searching multiple times, and continuing to search or not. Approach: Or maybe if you search and you don't have the answer, that's a bad thing to search and you need to go more specific / ask a different question!
        self.system_prompt_automatic_curriculum = f'''You are a helpful assistant that tells me the next immediate task to do. My ultimate goal is to discover as many useful pieces of information as possible to better achieve the research goal, answer as many questions as possible to get the best answer, and become the best researcher in the world in solving this research goal.

Research Goal: {self.research_problem}

I will give you the following information:
Files: these are my current files and skills that I have in my working directory.
Skills: these are skills that I can take action with.
Completed tasks so far (most recent to least): ...
Failed tasks that are too hard (most recent to least): ...
Most recent attempted tasks, plans, results, files, and answer states (newest to oldest): Answer states are the report of the best answer I have so far to achieving the research goal, and the attempted tasks, plans, results, and files are the tasks, plans, results, and files I took and had at that point in time to update the answer state.

1) You should act as a mentor and guide me to the next task based on my current learning progress. Always give me the task that will help me learn the most and reach my research goal the fastest. 
2) Please be very specific about what information or actions I need to take and what expected results I need to achieve.
3) The next task should follow a clear format, such as "Write [file]", "Reflect on why I'm seeing [error]", "Read [file]", "Brainstorm if ideas from [topic 1] be applied to [topic 2]", "Analyze what are the similarities between [topic 1] for success and [topic 2]" , "Reflect on what's significant about this paper: [paper]?", "Reflect on what's currently missing or preventing me from achieving [goal] better", etc. It should be a single task to collect useful information on. Do not propose multiple tasks at the same time. Do not mention anything else. 
4) The next task should not be too hard since the internet and I may not contain the full answer in a single article or have learned enough information to complete it yet. 
5) The next task should be novel and interesting based on my current learning progress. I should look for rare and potentially useful pieces of information, upgrade my current answer using better information, and discover new things. I should not be doing the same thing over and over again.
6) I may sometimes need to repeat some tasks or variations of the task if I need to collect more information to answer more difficult tasks. Only repeat tasks if necessary. 
7) I want to explore the world and discover new things. I don’t want to stay with my current answer for too long. 
8) Tasks that require information beyond another reader's ability to theoretically verify and reason if completed or correct should be avoided. For instance, "what else is there on the website?" and "what images and tables are on the website" are not ideal since they require visual confirmation from the screen. All the testing, coding, and asking other people questions should be avoided. Do not propose a task  with these keywords. You should only respond in the format as described below:

RESPONSE FORMAT: 
```json
{{ 
    "reasoning": "<based on the information I listed above, do reasoning about what the next task should be.>",
    "task": "<the next task.>"
}}
```

Here’s an example response: 
```json
{{ 
    "reasoning": "We know that we have a sword and we know there's fire, and fire lights things on fire. Therefore, we could try to make a firesword.",
    "task": "Try to make a firesword and record what happens."
}}
```

Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc. This is important.
'''

        # TODO: This is optional, might be useful, but to focus on a system prompt of asking questions and answering questions.
        # System 2: this is a more scoped down version where we have the focus be on only answering questions -- reading and analyzing information & asking questions. No action items. 
        # The current above system 1 is better for self-driving labs type of work where there are going to be more tasks.

    def get_exploration_progress(self):
        # TODO: this should contain inventory of where we're at now and what files we have / memory stream
        return f'''Completed tasks: {self.completed_tasks}, Failed tasks: {self.failed_tasks}'''

    def propose_next_task(self):
        '''
        This function decomposes a goal into tasks
        '''        
        user_prompt = f'''Files: {self.files}
Skills: {list(self.available_actions.keys())}    
Completed tasks so far: {self.completed_tasks}
Failed tasks that are too hard: {self.failed_tasks}
Most recent attempted tasks, plans, results, files, and answer states (newest to oldest):
{self.formatted_answer_states()}''' # TODO: Should I add formatted_action_history which includes tactical steps that were taken?
        
        print("System prompt for generating curriculum: \n", self.system_prompt_automatic_curriculum, "\n User prompt: ", user_prompt)
        next_task_response = self.complete_text_openai(system_prompt=self.system_prompt_automatic_curriculum, user_prompt=user_prompt, json_required=True)
        print("Response: ", next_task_response)
        next_task = json.loads(next_task_response)["task"]
        return next_task

    def get_completed_tasks():
        pass

    def get_failed_tasks():
        pass

    def add_completed_task(self, task, methods_prompt, result):
        # TODO: probably we should record the entire answer state of files, action, output, and answer state? Or just action and output?
        self.completed_tasks.insert(0, task)

        # Experimenting with adding the task to a living skill library in workspace so the methods prompt can build off of the skills library. 

        # Asking GPT to write a short file name, and then write the task + methods prompt to the file.
        res = self.complete_text_openai(system_prompt="You are a helpful assistant that writes a file name of the given task where the file contains a plan for potentially how to achieve that task. The file name should be less than 50 chars. Do not include the extension for the file name, .txt will be automatically added to the end. Your response should be only the file name.", user_prompt=f"Task: {task}")
        sanitized_file_name = self.sanitize_filename(res)
        with open(self.work_dir + "/skill_library/" + sanitized_file_name, "w") as f:
            f.write(f"Task: {task}\n")
            f.write(f"\nInstructions: {methods_prompt}")

        # Considering maintaining a running skill library, but adding files is likely not the way to build a wiki otherwise there will likely be a lot of overlapping information? Or not unless they're actually used in the skill library?
        with open(self.work_dir.split("_branch")[0] + "/skill_library/" + sanitized_file_name, "w") as f:
            f.write(f"Task: {task}\n")
            f.write(f"\nInstructions: {methods_prompt}")

        # Update answer state
        self.update_answer_state(task, methods_prompt, result)

    def add_failed_task(self, task, methods_prompt, result):
        self.failed_tasks.insert(0, task) #  + " \nCritique for why it failed: " + critique -- commented this out for now to allow for all tasks to be considered by the curriculum agent without truncation

        # Update answer state
        self.update_answer_state(task, methods_prompt, result)

    def sanitize_filename(self, text):
        # Remove invalid file name characters, replace spaces with underscores, lowercase and trim to 50 characters
        sanitized = re.sub(r'[^\w\s-]', '', text)  # Remove non-word characters except for spaces and hyphens
        sanitized = re.sub(r'\s+', '_', sanitized).strip()[:50].lower()  # Replace spaces, trim, and lower case
        return sanitized + '.txt'


curriculum_agent = CurriculumAgent(env)
next_task = curriculum_agent.propose_next_task()

System prompt for generating curriculum: 
 You are a helpful assistant that tells me the next immediate task to do. My ultimate goal is to discover as many useful pieces of information as possible to better achieve the research goal, answer as many questions as possible to get the best answer, and become the best researcher in the world in solving this research goal.

Research Goal: Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Evaluation
Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of

In [5]:
class SkillManager(Agent):
    def __init__(self, env):
        super().__init__(env)

    def retrieve_skills(self, task, execution_feedback):
        # For sake of simplicity, use recency for now (later relevancy and importance can be added). TODO: leverage task and execution feedback to guide what action_history memories to retrieve
        func_name_description_list = list(self.available_actions.keys())
        return func_name_description_list
    
    # Taking this out for now to just leave the base actions intact because other actions like training a model will be a file that can be retrieved as part of memory for the curriculum agent and not the skill agent
    # def retrieve_info_blocks(self, task, execution_feedback):
    #     # retrieving file names and description
    #     return [name + " - " + description for name, description in self.files]

    # Helper function: be able to create a function with a dynamic name and return value
    # def create_skill_function(self, function_name, return_value):
    #     def dynamic_method(self):
    #         return return_value
    #     # Bind the function to the instance as a method
    #     bound_method = types.MethodType(dynamic_method, self)
    #     setattr(self, function_name, bound_method)
    #     # Add the method to available functions
    #     self.available_functions[function_name] = bound_method

    # Core function: adding a new skill requires an original task, a validated answer, and a message history
    # def add_skill(self, task, validated_answer, methods_prompt):
    #     # TODO: wait until the action agent generates a function because maybe you only need to write a description of the input function instead of task and validated answer.
    #     # create_function_description_system_prompt = f'''You are a helpful assistant that writes a description of the given '''

    #     print("Adding skills! ", task, validated_answer, methods_prompt)

    #     create_skill_system_prompt = f'''You are a helpful assistant. Your goal is to write a short file name and a short description of the task and answer. 
        
    #     You will receive this information:
    #     Original task or question: ...
    #     Answer: ...

    #     Do not use any of these file names: {[name for name, _ in self.files]}

    #     Your output should be in the following format if function requires arguments:
    #     ```json
    #     {{
    #         "name": "<file_name>",
    #         "description": "<insert question and answer>"
    #     }}
    #     ```

    #     Good example output:
    #     ```json
    #     {{
    #         "name": "num_dogs_in_bens_family",
    #         "description": "The question was how many dogs are in the family. Ben said that he has 2 dogs in his family."
    #     }}
    #     ```

    #     Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc. This is important.
    #     '''

    #     create_function_description_prompt = f'''
    #     Original task or question: {task}
    #     Answer: {validated_answer}
    #     '''
    #     res, messages = complete_text_openai(prompt=create_function_description_prompt, system_prompt=create_skill_system_prompt, verbose=True)
    #     res

    #     try:
    #         # Load the function description
    #         file_name_description = json.loads(res['content'])
    #         print("file_name_description: ", file_name_description)

    #         # Create the function as a method of skill_manager
    #         self.write_file(file_name_description['name'], f"Question: {task}\nAnswer: {validated_answer}\nReasoning and Methods: {methods_prompt}")

    #         # Add function to function description list
    #         self.files.append((file_name_description['name'], file_name_description['description']))

    #         print("COMPLETE!")
            
    #     except Exception as e:
    #         print(f"An unexpected error occurred: {e}")
    #         return

    #     return
    
skill_manager = SkillManager(env)
print("Available functions: \n", skill_manager.tool_descriptions, "\n\nFunctions", list(skill_manager.available_actions.keys()), "\n\nAvailable files: \n", skill_manager.files)

Available functions: 
 [{'type': 'function', 'function': {'name': 'reflection', 'description': 'Use this to reflect on all past steps. Provide a detailed description on what to reflect on and what should be returned.', 'parameters': {'type': 'object', 'properties': {'things_to_reflect_on': {'type': 'string', 'description': 'A detailed description on what to reflect on and what should be returned'}}, 'required': ['things_to_reflect_on']}}}, {'type': 'function', 'function': {'name': 'readFile', 'description': 'Use this to read an existing file.', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'A valid file name with relative path to current directory if needed'}}, 'required': ['file_name']}}}, {'type': 'function', 'function': {'name': 'writeFile', 'description': 'Use this to write a file. If the file already exists, it will be overwritten.', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'A va

In [6]:
class MethodsAgent(Agent):
    # TODO: does there need to be an action agent generating the steps? Or does there need to be a separate execution agent running the prompt? Or can the action agent be the execution agent?
    # TODO: can a critic agent really check if the output is correct? Or can they only check if that aligns with expectation? Otherwise, the critic will have to check the line of content values to make sure the reasoning is sound, which is still doable, but the extent that another critic can check is limited. I guess it's just to make sure the reasoning is sound.

    def __init__(self, env):
        super().__init__(env)

    def generate_function_callable_prompt(self, task,
                methods_prompt,
                execution_feedback,
                execution_errors,
                critique,
                skills):
        generate_plan_system_prompt = '''You are a helpful assistant and a first-rate problem solver. Given a task or question, your goal is to list out the steps to solve that task given your skills and reasoning. Ultimately, your output should be able to be followed by a human limited by the skills and knowledge given, and another human should be able to check that human's output to see if it's correct and reasonable. Note that the functions asked for may sometimes already be called and the information from the function that you need is already in the prompt, so read carefully. Note that you DO NOT have the ability to see, you can only read, think, write, and execute scripts using the existing skills and knowledge.

You will be given this information:
Task or question: ...
Skills: these are skills that I can take action with.
Files: these are my current files that I have in my working directory.  
Current state plan: ...
Current state output after executing steps: ...
Execution errors: ...
Critique: ...
History of files, action, and result (newest to oldest): By following the plan, this is my history of files, action, and result I had and took at that point in time. 

You should then respond to me with
Explain (if applicable): Are there any steps missing in your plan? Why do the current state steps not complete the task? What do the current state output, execution errors, and critique imply?
Plan: How to complete the task step by step. You should pay attention and read Files because it tells you what information you have. The task completeness check is also based on your final action results and final files.
Steps: 
1) Write how to complete the task step by step. 
2) Reuse the above useful files as much as possible.
3) Your task completion and plan will be reused to achieving more complex tasks. Thereofre, you should make it generic and reusable. You should not make strong assumptions about the files (as it may be changed at a later time), and therefore you should always check whether you have the required files before using them. If not, you should first create the required files, get the necessary information, and reuse the above useful actions.
'''

        user_prompt = f'''Task: {task}
Files: {self.files}
Skills: {list(self.available_actions.keys())}  
Skills: {skills}
Current state plan: {methods_prompt}
Current state output after executing steps: {execution_feedback}
Execution errors: {execution_errors}
Critique: {critique}
History of files, action, and result:
{self.formatted_action_history()}'''

        methods_agent_feedback = self.run_assistant(system_prompt=generate_plan_system_prompt, user_prompt=user_prompt, tool_descriptions=self.read_tool_description)
        print("methods_agent_feedback", methods_agent_feedback)

        return methods_agent_feedback
    
methods_agent = MethodsAgent(env)
methods_prompt = None
execution_feedback = None
execution_errors = None
critique = None
success = False
methods_prompt = methods_agent.generate_function_callable_prompt(task=next_task, methods_prompt=methods_prompt, execution_feedback=execution_feedback, execution_errors=execution_errors, critique=critique, skills=list(skill_manager.available_actions.keys()))


run.status:  queued

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  completed
methods_agent_feedback Explain: The current state does not complete the task because there has been no action taken yet to read 'data_description.txt'. The task at hand is to read the file and summarize the key points, which has been correctly identified by the previous reasoning, but the steps to achieve this have not been executed.

Plan:
1. Check if 'data_description.txt' exists in the list of available files.
2. If the file exists, use the 'readFile' function to read the content of 'data_description.txt'.
3. Summarize the key points from the file in a human-readable format.

Steps:
1) Check if 'data_description.txt' is present in the current list of files.
2) If the file is present, call the 'readFile' function with 'data_description

In [7]:
class ExecutionAgent(Agent):
    # Ultimately the answer will go into Description or be "returned" with the description being condensed into a tldr and the methods_prompt will be added to history

    # TODO: Perhaps the execution agent can get info about the files too, but maybe that's the action agent's responsibility.
    def __init__(self, env):
        super().__init__(env)

    def function_call(self, task, methods_prompt):
        system_prompt = f'''You are a helpful assistant. Your goal is to execute the given instructions and output the complete answer to the question. If the instructions don't seem reasonable or you cannot get to the complete answer, then you should give feedback on why you couldn't do it and what you tried. 

You will be given this information:
Skills: these are skills that I can take action with.
Files: these are my current files that I have in my working directory.
Task: ...
Instructions: ...
History of files, action, and result (newest to oldest): By following the plan, this is my history of files, action, and result I had and took at that point in time.'''
        execute_prompt = f'''Skills: {list(self.available_actions.keys())}
Files: {self.files}
Task: {task}
Instructions: {methods_prompt}
History of files, action, and result:
{self.formatted_action_history()}'''

        try:
            # complete_text_openai(system_prompt=system_prompt, prompt=execute_prompt, tools=self.tool_descriptions, available_functions=self.available_actions) # Normal function calling
            self.run_assistant(system_prompt=system_prompt, user_prompt=execute_prompt) # OpenAI Assistants API
            pass
        except Exception as e:
            return ""
        
        return self.formatted_action_history() # Difficult to manage a start index for only answers because some actions get popped or repeated
execution_agent = ExecutionAgent(env)

In [8]:
execution_feedback = execution_agent.function_call(methods_prompt=methods_prompt, task=next_task)
print("Execution feedback", execution_feedback)


run.status:  in_progress

run.status:  in_progress

run.status:  requires_action
Run required action: 
tool_id: call_2NgB8f6FyeR0cimYnAfhXekP, 
tool_function.arguments: {"file_name":"data_description.txt"}, 
tool_function.name: readFile, 
tool_type: function


--- LOGGING NEW ACTION ---
Step: 1
Calling function wrapped_read_file(args = (), kwargs = {'file_name': 'data_description.txt', 'work_dir': 'workspace\\home-data-for-ml-course_branch'})

--- TOOL SUCCESS ---

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.status:  in_progress

run.statu

In [9]:
class CriticAgent(Agent):
    # TODO: Perhaps include the skills to the critic so the critic knows the facts to check if this makes sense or not
    def __init__(self, env):
        super().__init__(env)

    def check_task_success(self, task, methods_prompt, execution_feedback):
        # took out "Approach: My plan and reasoning to achieve the task." because critic agent would say Success to a good plan but incomplete reuslts
        system_prompt = '''You are a first-rate researcher that assesses my progress of research and provides useful guidance. 
        
Based on the final files, actions, and results, you are required to evaluate if I have already completed and satisfied all the task requirements. Exceeding the task requirements is also considered a success while failing to complete any of them requires you to provide critique to help me improve and mark my success as False. There must be evidence to show that all the task requirements are already and fully completed for it to be counted as a success. This is important.

I will give you the following information:
Skills: these are skills that I can take action with.
Files: these are my current files that I have in my working directory.
Task: The objective I need to accomplish.
History of files, action, and result (newest to oldest): After following the plan, this is my history of files, action, and result I had and took at that point in time.

You should only respond in JSON format as described below:
```json
{
    "task": "task",
    "evidence": "potential evidence of success",
    "counter_evidence": "potential evidence of failure",
    "reasoning": "reasoning",
    "success": boolean,
    "critique": "critique",
}
```
Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc.
'''
# Commenting out the example because GPT3.5 just inappropriately uses it verbatim it sometimes
# RESPONSE:
# {
#     "reasoning": "The reasoning to get to the answer makes sense, but there's no direct answer for what the actual distribution of the sale price is.",
#     "success": False,
#     "critique": "The answer only tells us how to get the distribution is, but does not tell us what the actual distribution. Please tell us what the actual distribution is.",
# }

        user_prompt = f'''You can only read files to help check if task has been fully completed.
Skills: {list(self.available_actions.keys())}
Files: {self.files}
Task: {task}
History of files, action, and result: {self.formatted_action_history()}''' # Execution feedback should be logged in self.formatted_action_history()

        # 1. Employing a read assistant first to log files to be checked into file_action_result_history for the critic agent
        response_message = self.run_assistant(system_prompt=system_prompt, user_prompt=user_prompt, tool_descriptions=self.read_tool_description)

        # 2. Employing a chat completion based on the updated file_action_reuslt_history to make a final judgement
        user_prompt = f'''Skills: {list(self.available_actions.keys())}
Files: {self.files}
Task: {task}
History of files, action, and result: {self.formatted_action_history()}''' # Execution feedback should be logged in self.formatted_action_history()
        
        print("Critic system prompt: ", system_prompt, "\n\nCritic user prompt: ", user_prompt, "\n\nTask: " + task + "\n\nCritic response: ", response_message)

        response_message = self.complete_text_openai(system_prompt=system_prompt, user_prompt=response_message, json_required=True)

        try:
            response_json = json.loads(response_message)
            success = response_json['success'] # Must be bool
            evidence = str(response_json['evidence'])
            opposition = str(response_json['counter_evidence'])
            reasoning = str(response_json['reasoning'])
            critique = str(response_json['critique'])
        except Exception as e:
            return False, response_message + " JSON parsing error: " + str(e)

        # Handle null values
        if not reasoning:
            reasoning = ""
        if not critique:
            critique = ""

        return success, "\nEvidence: " + evidence + "\nCounter evidence: " + opposition + "\nReasoning: " + reasoning + "\nCritique: " + critique
critic_agent = CriticAgent(env)

In [10]:
success, critique = critic_agent.check_task_success(task=next_task, methods_prompt=methods_prompt, execution_feedback=execution_feedback)
print("Success: ", success, "\nCritique: ", critique)

Assistants Truncated user prompt:  You can only read files to help check if task has been fully completed.
Skills: ['reflection', 'readFile', 'writeFile', 'executeScript']
Files: ['data_description.txt', 'research_problem.txt', 'sample_submission.csv', 'sample_submission.csv.gz', 'test.csv', 'test.csv.gz', 'train.csv', 'train.csv.gz']
Task: Read 'data_description.txt' and summarize the key points.
History of files, action, and result: 
Step: 2
Files: ['data_description.txt', 'research_problem.txt', 'sample_submission.csv', 'sample_submission.csv.gz', 'test.csv', 'test.csv.gz', 'train.csv', 'train.csv.gz']
Action: Calling function wrapped_read_file(args = (), kwargs = {'file_name': 'data_description.txt', 'work_dir': '.'})
Result: MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINI

In [11]:
# # START HERE FOR NEXT ITERATIONS: Save the current iteration's state after this cycle in case we need to revert
# skill_manager_copy = copy.deepcopy(skill_manager)
# curriculum_agent_copy = copy.deepcopy(curriculum_agent)

In [12]:
# # REVERT BY STARTING HERE, uncomment the below and run
# skill_manager = copy.deepcopy(skill_manager_copy)
# curriculum_agent = copy.deepcopy(curriculum_agent_copy)

In [13]:
# Running a cycle of Voyager
num_rounds = 2 # Just to test if it can give easier tasks too
num_tasks = 25

env = Environment(args)
curriculum_agent = CurriculumAgent(env)
skill_manager = SkillManager(env)
methods_agent = MethodsAgent(env)
execution_agent = ExecutionAgent(env)
critic_agent = CriticAgent(env)

Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/house-price-testing_voyager-v4_gpt4_v2', work_dir='workspace', max_steps=50, max_time=18000, device=0, python='/home/user/micromamba/envs/autogpt/bin/python', interactive=False, resume=None, resume_step=0, agent_type='SimpleAssistantAgent', llm_name='gpt-4-1106-preview', fast_llm_name='gpt-4-1106-preview', edit_script_llm_name='gpt-4-1106-preview', edit_script_llm_max_tokens=4000, agent_max_steps=50, actions_remove_from_prompt=[], actions_add_to_prompt=[], no_retrieval=False, valid_format_entires=None, max_steps_in_context=3, max_observation_steps_in_context=3, max_retries=4, langchain_agent='zero-shot-react-description')
Preparing task home-data-for-ml-course , of type:  kaggle


In [14]:
for task_idx in range(num_tasks):
    print(f"\nTask {task_idx} of {num_tasks}")

    exploration_progress = curriculum_agent.get_exploration_progress()
    print("Exploration progress: ", exploration_progress)

    next_task = curriculum_agent.propose_next_task()
    print("next_task", next_task)

    methods_prompt = next_task # First round, methods prompt is the task
    execution_feedback = None
    execution_errors = None
    critique = None
    success = False
    for i in range (num_rounds):
        print(f"\nRound {i} Task: ", next_task)

        if i != 0: # Don't need to generate methods prompt for the first round
            print("\nStarting methods agent")
            methods_agent = MethodsAgent(env)
            methods_prompt = methods_agent.generate_function_callable_prompt(task=next_task, methods_prompt=methods_prompt, execution_feedback=execution_feedback, execution_errors=execution_errors, critique=critique, skills=list(skill_manager.available_actions.keys()))
            print("\nMethods agent output:\n", methods_prompt)

        print("\nStarting execution agent")
        execution_agent = ExecutionAgent(env)
        execution_feedback = execution_agent.function_call(methods_prompt=methods_prompt, task=next_task)
        print("\nExecution agent output: ", execution_feedback)

        print("\nStarting critic agent")
        critic_agent = CriticAgent(env)
        success, critique = critic_agent.check_task_success(task=next_task, methods_prompt=methods_prompt, execution_feedback=execution_feedback)
        print("Critic agent output", "\nSuccess: ", success, "\nCritique: ", critique)

        if success:
            break
    if success:
        curriculum_agent.add_completed_task(next_task, methods_prompt, critique)
    else:
        curriculum_agent.add_failed_task(next_task, methods_prompt, critique)


Task 0 of 25
Exploration progress:  Completed tasks: [], Failed tasks: []
System prompt for generating curriculum: 
 You are a helpful assistant that tells me the next immediate task to do. My ultimate goal is to discover as many useful pieces of information as possible to better achieve the research goal, answer as many questions as possible to get the best answer, and become the best researcher in the world in solving this research goal.

Research Goal: Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Evaluation
Goal
It is your job to predict the sales price

KeyboardInterrupt: 